In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

In [ ]:
covariates = pd.read_csv("/rds/general/user/meb22/projects/ukbiobank/live/ukbiobank/data_2025/proteomics/Processed_all_covariates.csv").iloc[:,1:]


In [ ]:
covariate_cols = ["Age","Sex","Ethnicity","BMI","Season","fasting_time","sample_age","smoking","alcohol"]

In [ ]:
df_all_pivot_training = pd.read_csv("Training_data_imputed_lesscols.csv")

In [ ]:
df_all_pivot_traininghc= pd.read_csv("Training_healthycontrol.csv")

In [ ]:
df_all_cov = pd.merge(df_all_pivot_training, covariates, on="eid")

In [ ]:
healthy_controls_training = df_all_pivot_traininghc[df_all_pivot_traininghc["Diagnosis"] ==0]["eid"].unique().tolist()

In [ ]:
df_all_pivot_training= df_all_pivot_training[df_all_pivot_training["eid"].isin(healthy_controls_training)]

In [ ]:
df_all_cov_dropna = df_all_cov.dropna(subset=covariate_cols)

In [ ]:
df_all_cov_dropna.set_index(["eid"],inplace=True)

In [ ]:
df_all_cov_dropna_all = df_all_cov_dropna.copy(deep=True)

In [ ]:
df_all_cov_dropna

In [ ]:
df_all_cov_dropna = df_all_cov_dropna[df_all_cov_dropna.index.isin(healthy_controls_training)]

In [ ]:
df_all_cov_dropna_rest = df_all_cov_dropna_all[~df_all_cov_dropna_all.index.isin(healthy_controls_training)]


In [ ]:
df_all_cov_dropna_rest

In [ ]:
protein_cols = df_all_cov_dropna.columns[:-16]

In [ ]:
protein_cols

In [ ]:
df_all_cov_dropna

In [ ]:
import statsmodels.formula.api as smf
import joblib

model_params = {}
residual_data = {}  # temporary dict to collect residuals

for protein in protein_cols:
    # Construct formula terms
    terms = [f"C({col})" if df_all_cov_dropna[col].dtype == "object" else col for col in covariate_cols]

    lhs = f'Q("{protein}")'
    rhs = ' + '.join(terms)
    formula = f'{lhs} ~ {rhs}'

    # Fit model
    model = smf.ols(formula, data=df_all_cov_dropna).fit()

    # Collect residuals
    residual_data[protein] = model.resid

    # Store model info
    model_params[protein] = {
        'coeffs': model.params,
        'covariates_used': covariate_cols,
        'terms': terms,
        'model_formula': formula,
        'categorical_levels': {
            col: df_all_cov_dropna[col].astype("category").cat.categories.tolist()
            for col in covariate_cols if df_all_cov_dropna[col].dtype == 'object'
        }
    }


In [ ]:
residuals_df_all = pd.DataFrame(residual_data, index=df_all_cov_dropna.index)

# Save model and residuals
joblib.dump(model_params, "ancova_modelshealthycontrol.pkl")


In [ ]:
residuals_df_all

In [ ]:
flat_params = []

for protein, details in model_params.items():
    for covariate, coef in details['coeffs'].items():
        flat_params.append({
            "protein": protein,
            "covariate": covariate,
            "coefficient": coef,
            "model_formula": details["model_formula"]
        })

pd.DataFrame(flat_params).to_csv("ancova_modelshealthycontrol.csv", index=False)


In [ ]:
ancova_coef =pd.read_csv("ancova_modelshealthycontrol.csv")

In [ ]:
coef_pivot = ancova_coef.pivot(index='covariate', columns='protein', values='coefficient')

In [ ]:
import patsy

predicted_data = {}  # Collect predictions for each protein here

# Copy the data to avoid mutating your original DataFrame
df_temp = df_all_cov_dropna_rest.copy()

for protein, info in model_params.items():
    formula = info["model_formula"]
    coeffs = info["coeffs"]

    # Set categorical levels to match training
    if 'categorical_levels' in info:
        for col, levels in info['categorical_levels'].items():
            if col in df_temp.columns:
                df_temp[col] = pd.Categorical(df_temp[col], categories=levels)

    # Create design matrix
    try:
        design_matrix = patsy.dmatrix(
            formula.split("~")[1],
            df_temp,
            return_type='dataframe'
        )
    except Exception as e:
        print(f"Error generating design matrix for {protein}: {e}")
        continue

    # Add missing columns as 0
    for col in coeffs.index:
        if col not in design_matrix.columns:
            design_matrix[col] = 0

    # Reorder columns to match model
    design_matrix = design_matrix[coeffs.index]

    # Predict
    predicted_data[protein] = design_matrix @ coeffs

# Create DataFrame all at once
predicted_df = pd.DataFrame(predicted_data, index=df_all_cov_dropna_rest.index)


In [ ]:
residuals_df = df_all_cov_dropna_rest[protein_cols] - predicted_df

In [ ]:
residuals_df_all_all = pd.concat([residuals_df_all, residuals_df])

In [ ]:
residuals_df_all_all.to_csv("Residualised_healthycontrol_training_all.csv")

In [ ]:
residuals_df_all_all

In [ ]:
test_data = pd.read_csv("Testing_data_imputed_lesscols.csv")

In [ ]:
df_test = pd.merge(test_data.reset_index(), covariates, on="eid")

In [ ]:
df_test.set_index(["eid"],inplace=True)

In [ ]:
import patsy

predicted_data = {}  # Collect predictions for each protein here

# Copy the data to avoid mutating your original DataFrame
df_temp = df_test.copy()

for protein, info in model_params.items():
    formula = info["model_formula"]
    coeffs = info["coeffs"]

    # Set categorical levels to match training
    if 'categorical_levels' in info:
        for col, levels in info['categorical_levels'].items():
            if col in df_temp.columns:
                df_temp[col] = pd.Categorical(df_temp[col], categories=levels)

    # Create design matrix
    try:
        design_matrix = patsy.dmatrix(
            formula.split("~")[1],
            df_temp,
            return_type='dataframe'
        )
    except Exception as e:
        print(f"Error generating design matrix for {protein}: {e}")
        continue

    # Add missing columns as 0
    for col in coeffs.index:
        if col not in design_matrix.columns:
            design_matrix[col] = 0

    # Reorder columns to match model
    design_matrix = design_matrix[coeffs.index]

    # Predict
    predicted_data[protein] = design_matrix @ coeffs

# Create DataFrame all at once
predicted_df = pd.DataFrame(predicted_data, index=df_test.index)


In [ ]:
residuals_df = df_test[protein_cols] - predicted_df

In [ ]:
residuals_df.to_csv("Testing_data_residualshealthycontrol.csv")